In [ ]:
%matplotlib qt5

import os
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as mplt

from pymodels import si
import pyaccel as pa
from mathphys.functions import load, save
from apsuite.loco import LOCOAnalysis
from apsuite.orbcorr import OrbRespmat

from siriuspy.clientconfigdb import ConfigDBClient

mplt.rcParams.update({
    'font.size': 18, 'axes.grid': True, 'grid.alpha': 0.5, 'grid.linestyle': '--',
    'grid.linewidth': 1, 'lines.linewidth': 2,
})

# Define functions

In [ ]:
def get_idcs(fam_name, mod, idc_id):
    idc = np.array(pa.lattice.find_indices(mod, 'fam_name', fam_name))
    idx = np.argsort(np.abs(idc_id-idc))[:2]
    return idc[idx]


def calc_optics_respm(mod, beamline='EMA'):
    ivu_idcs = pa.lattice.find_indices(mod, 'fam_name', 'IVU18')
    if beamline == 'EMA':
        idc_id = ivu_idcs[0]
    elif beamline == 'PAINEIRA':
        idc_id = ivu_idcs[2]

    idc_qn = []
    idc_qn.append(get_idcs('QFB', mod, idc_id))
    idc_qn.append(get_idcs('QDB1', mod, idc_id))
    idc_qn.append(get_idcs('QDB2', mod, idc_id))
    idc_qn = np.array(idc_qn)
    knob_names = ['QFB', 'QDB1', 'QDB2']

    respm = OrbRespmat(mod, 'SI', dim='6d', corr_system='SOFB')
    mat0 = respm.get_respm()

    opt_resp = []
    dkl = 0.001
    for idcs in idc_qn:
        kl0 = np.array(pa.lattice.get_attribute(respm.model, 'KL', idcs))
        pa.lattice.set_attribute(respm.model, 'KL', idcs, kl0 + dkl)

        dmat = respm.get_respm() - mat0
        dmat[:, -1] *= 1e6  # convert from m/hz to um/Hz
        dmat /= dkl
        opt_resp.append(dmat.ravel())
        pa.lattice.set_attribute(respm.model, 'KL', idcs, kl0)

    return np.array(opt_resp), idc_qn, knob_names


def get_all_loco_fitting_folders(folder, prefix=''):
    folder += '' if folder.endswith('/') else '/'
    return sorted([
        folder + f for f in os.listdir(prefix + folder) if '.' not in f and 'IVU_EMA_ACORM' in f])


# Load data

In [ ]:
prefix = './'
allfols = []
allfols += get_all_loco_fitting_folders('loco_input', prefix=prefix)

kparams = np.array([float(fname.split('/')[-1].split('_')[-1].replace('p', '.')) for fname in allfols])
runs = np.array([fname.split('/')[0] for fname in allfols])

In [ ]:
allfols

In [ ]:
# Construct loco analysis
alllocoanls = []
for fol in allfols:
    parent, fol = os.path.split(fol)
    anl = LOCOAnalysis(
        fname_setup=prefix + f'{parent}/loco_input_{fol}.pickle',
        fname_fit=prefix + f'{parent}/{fol}/fitting_{fol}.pickle',
    )
    anl.get_setup()
    alllocoanls.append(anl)

# Check coupling

In [ ]:
# Get emittances and coupling from loco analysis

alldf_emits = []
for fol, anl in zip(allfols, alllocoanls):
    print(fol)
    anl.get_loco_results()
    alldf_emits.append(anl.emittance_and_coupling())

emit = np.array([e['LOCO model'][1] for e in alldf_emits])
tune = np.array([e['LOCO model'][3] for e in alldf_emits])

# Get CAX beam parameters variations

In [ ]:
# Get equilibrium parameters from fitted model
alleqparams = []
for fol, anl in zip(allfols, alllocoanls):
    print(fol)
    anl.get_loco_results()
    alleqparams.append(pa.optics.EqParamsFromBeamEnvelope(
            anl.loco_fit['fit_model']))


In [ ]:
nom_mod = si.create_accelerator()
idc = pa.lattice.find_indices(nom_mod, 'fam_name', 'B1_SRC')[0]
sigy = np.array([e.sigma_rx[idc] for e in alleqparams])
tilt = np.array([e.tilt_xyplane[idc] for e in alleqparams])


# Check LOCO tunes

In [ ]:
alltwiss = []
for fol, anl in zip(allfols, alllocoanls):
    print(fol)
    anl.get_loco_results()
    model = anl.loco_fit['fit_model']
    twiss, *_ = pa.optics.calc_twiss(model, indices='open')
    alltwiss.append(twiss)

In [ ]:
ref_idx = allfols.index('loco_input/IVU_EMA_ACORM_kparam_24p000')
ref_twiss = alltwiss[ref_idx]
tunex0 = ref_twiss.mux[-1]/(2*np.pi)
tuney0 = ref_twiss.muy[-1]/(2*np.pi)
alltunex = []
alltuney = []
for twiss in alltwiss:
    alltunex.append(twiss.mux[-1]/(2*np.pi) - tunex0)
    alltuney.append(twiss.muy[-1]/(2*np.pi) - tuney0)
alltunex = np.array(alltunex)
alltuney = np.array(alltuney)

# Correct optics with local quadrupoles

In [ ]:
allorms = []
allquad_stren = []

print('Getting nominal model...')
ref_idx = allfols.index('loco_input_run/IVU_EMA_ACORM_kparam_24p000')
anl_ref = alllocoanls[ref_idx]
refmat = anl_ref.loco_setup['respmat']
nom_mod, *_ = anl_ref.get_nominal_model()
print('Done.')
print('Calculating optics response matrix...')
opt_resp, idc_qn, knob_names = calc_optics_respm(nom_mod)
print('Done.')

In [ ]:
u, s, vt = np.linalg.svd(opt_resp.T, full_matrices=False)
s =1/s
imat = np.dot(vt.T*s, u.T)

mplt.plot(vt[0, :], '-o')
mplt.plot(vt[1, :], '-o')
mplt.plot(vt[2, :], '-o')
mplt.plot(vt[3, :], '-o')

In [ ]:
for fol, anl in zip(allfols, alllocoanls):
    orm = anl.loco_setup['respmat']
    allorms.append(orm)
    orm = orm - refmat

    # quads = 1*np.dot(imat, orm.ravel())
    quads, *_ = np.linalg.lstsq(opt_resp.T, orm.ravel(), rcond=None)
    allquad_stren.append(quads)

allquad_stren = np.array(allquad_stren)
allorms = np.array(allorms)